## Ingestion del archivo "genre.csv"

#### `Paso 0 - Actualización de variables y funciones

In [0]:
dbutils.widgets.text("p_environment", "")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

#### Paso 1 - Leer el archivo CSV usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DataType

In [0]:
#%fs
#ls /mnt/moviehistoryadilmor/bronze

In [0]:
#display(spark.read.option("header", True).csv("/mnt/moviehistoryadilmor/bronze/genre.csv"))

In [0]:
genre_schema = StructType(fields=[
    StructField("genreId", IntegerType(), False), \
    StructField("genreName", StringType(), True) \
])

In [0]:
genre_df = spark.read \
    .option("header", True) \
    .schema(genre_schema) \
    .csv(f"{bronze_folder_path}/{v_file_date}/genre.csv")

In [0]:
#display(genre_df)

#### Paso 2 - Seleccionar sólo las columnas "requeridas"

In [0]:
from pyspark.sql.functions import col

genre_selected_df = genre_df.select(
                            col("genreId"),
                            col("genreName"))

#### Paso 3 - Cambiar el nombre de las columnas según lo "requerido"

In [0]:
# Alterando somente os nomes das colunas necessárias
genre_renamed_df = genre_selected_df \
                    .withColumnRenamed("genreID", "genre_id") \
                    .withColumnRenamed("genreName", "genre_name")

#### Paso 4 - Agregar la columna "ingestion_date" al DataFrame

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
genre_final_df = add_ingestion_date(genre_renamed_df) \
                .withColumn("enviroment", lit(v_environment)) \
                .withColumn("file_date", lit(v_file_date))

#### Paso 5 - Escribir datos en el datalake en formato parquet

In [0]:
#genre_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/genre")
#genre_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.genre")
genre_final_df.write.mode("overwrite").format("delta").saveAsTable("movie_silver.genre")

In [0]:
#%fs
#ls /mnt/moviehistoryadilmor/silver/genre

In [0]:
#display(spark.read.parquet(f"{silver_folder_path}/genre"))
#spark.sql("SELECT * FROM movie_silver.genre").display()

In [0]:
dbutils.notebook.exit("Success")